In [1]:
from agc.dataset import AtariDataset
import numpy as np
from os import path, listdir, remove
import cv2
from matplotlib import pyplot as plt
from shutil import copyfile, rmtree, move

%matplotlib inline

In [3]:
DATASET_PATH = '/media/HDD1/data/atari_v2/atari_v2_release/'
SCREENS_PATH = path.join(DATASET_PATH, 'screens')
TRAJS_PATH = path.join(DATASET_PATH, 'trajectories')

In [70]:
dataset = AtariDataset(DATASET_PATH)

In [61]:
def remove_traj_from_data(traj_path):
    '''Be careful! This will permanently remove the trajectory and its screenshots'''
    print('Removing %s and its screens' % traj_path)
    #conver traj path into screen path
    split_path = traj_path.split('/')
    traj_number = split_path[-1].split('.')[0]
    game_name = split_path[-2]
    screen_folder = path.join(SCREENS_PATH, game_name, traj_number)
    
    #remove the traj file
    remove(traj_path)
    
    #remove the screens
    rmtree(screen_folder)

In [63]:
# the replay process was interrupted several time
# just to be sure, that we do not have the duplicated trajectories
# first line has the db unique id of the traj
# this should be also unique in the dataset
db_ids = {}

for g in listdir(TRAJS_PATH):
    trdir = path.join(TRAJS_PATH, g)
    for t in listdir(trdir):
        traj_path = path.join(trdir, t)
        with open(traj_path, 'r') as f:
            lines = f.readlines()
            db_id = int(lines[0].split()[-1])
            if db_id not in db_ids.keys():
                db_ids[db_id] = []    
            db_ids[db_id].append(traj_path)

# uncomment to delete
# for i in db_ids:
#     if len(db_ids[i]) > 1:
#         # leave the first and remove the others
#         for tr in db_ids[i][1:]:
#             remove_traj_from_data(tr)
        
print('DONE') 


DONE


In [71]:
suspicious_trajs = []

for g in dataset.trajectories:
    trajs = dataset.trajectories[g]
    first_frames_scores = []
    ctr = 0
    for t in trajs:
        # 2 is for score  
        scores = [trajs[t][fr]['score'] for fr in range(10)]
        if np.sum(scores) > ctr:
            suspicious_trajs.append((g,t,scores))

# uncomment to remove
for t in suspicious_trajs:
    print(t)
    #tpath = path.join(TRAJS_PATH, t[0], str(t[1]) + '.txt')
    #remove_traj_from_data(tpath)

In [114]:
#fix replay bug (less pics than trajectories)
#pic numbering is from 1, traj steps from 0
#fixed in the webapp code already, but need to fix here not to replay again

# for g in listdir(SCREENS_PATH):
#     gdir = path.join(SCREENS_PATH, g)
#     for t in listdir(gdir):
#         #get int file names
#         dir_files = listdir(path.join(gdir, t))
#         dir_files.sort(key=lambda f: int(f.split('.')[0]))
#         #note, that without sort, this method will make a mess
#         for f in dir_files:
#             new_fname = str(int(f.split('.')[0]) - 1) + '.png'
#             move(path.join(gdir, t, f), path.join(gdir, t, new_fname))
# print('DONE')

DONE


In [123]:
ctr = 0
for g in listdir(SCREENS_PATH):
    gdir = path.join(SCREENS_PATH, g)
    for t in listdir(gdir):
        n_pics = len(listdir(path.join(gdir,t)))
        #check traj len now
        traj_path = path.join(TRAJS_PATH, g, t + '.txt')
        with open(traj_path, 'r') as f:
            lines = f.readlines()
        
        # 2 is for first two metadata lines
        if len(lines) < n_pics + 2: 
            print('OOPS, less data than needed for %s' % traj_path)
            print(n_pics, len(lines))
            assert False, "you should not be here"
           
        if len(lines) > n_pics + 2:
            #this is due to initial code collection bug
            #when we did not reset part of the env
            with open(traj_path, 'w') as f:
                f.writelines(lines[:n_pics + 2])
        
print('DONE') 

DONE


In [6]:
# remove final score zero trajectories

to_delete = []

for g in listdir(SCREENS_PATH):
    gdir = path.join(SCREENS_PATH, g)
    for t in listdir(gdir):
        traj_path = path.join(TRAJS_PATH, g, t + '.txt')
        with open(traj_path, 'r') as f:
            lines = f.readlines()
            final_line = lines[-1]
            final_score = int(final_line.split(',')[2])
            if final_score < 0:
                print('WTF')
            if final_score == 0:
                to_delete.append(path.join(g, t))

for t in to_delete:
    print(t)
    # remove traj
    remove(path.join(TRAJS_PATH, t + '.txt'))
    #remove screen folder for this traj
    rmtree(path.join(SCREENS_PATH, t))
print('DONE')

spaceinvaders/172
spaceinvaders/260
spaceinvaders/991
spaceinvaders/821
spaceinvaders/255
spaceinvaders/174
spaceinvaders/164
spaceinvaders/347
spaceinvaders/10
spaceinvaders/261
spaceinvaders/286
spaceinvaders/298
spaceinvaders/169
spaceinvaders/1029
spaceinvaders/175
spaceinvaders/114
spaceinvaders/486
spaceinvaders/974
spaceinvaders/168
spaceinvaders/170
spaceinvaders/487
spaceinvaders/173
spaceinvaders/479
spaceinvaders/171
spaceinvaders/166
spaceinvaders/485
qbert/766
qbert/1738
qbert/1800
qbert/763
qbert/1736
qbert/759
qbert/1792
qbert/761
qbert/765
qbert/1799
qbert/762
qbert/1739
qbert/1737
qbert/1798
qbert/758
qbert/760
qbert/1735
revenge/2635
revenge/2358
revenge/2632
revenge/725
revenge/14
revenge/971
revenge/2618
revenge/2617
revenge/1781
revenge/2521
revenge/2648
revenge/728
revenge/852
revenge/2495
revenge/2649
revenge/2625
revenge/2642
revenge/917
revenge/2623
revenge/2498
revenge/2636
revenge/2633
revenge/2631
revenge/2640
revenge/2531
revenge/2622
revenge/721
revenge/26